In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import tensorflow as tf
print(tf.__version__)

In [28]:
pip install tensorflow --upgrade

In [3]:
pip install livelossplot

In [4]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from tensorflow import keras

import cv2
import PIL
import os
import pathlib
import shutil
from IPython.display import Image, display

# Plotly for the interactive viewer (see last section)

import plotly.graph_objs as go
import plotly.graph_objects as go
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, Model,load_model
from tensorflow.keras.applications.vgg16 import VGG16,preprocess_input
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.xception import Xception, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
from tensorflow.keras import Input
from tensorflow.keras.callbacks import EarlyStopping, LearningRateScheduler 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten,BatchNormalization,Activation
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, Callback, EarlyStopping
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers.experimental import preprocessing

import gc
import skimage.io
from livelossplot import PlotLossesKeras

## Load Data

In [5]:
train_src = '../input/dog-breed-detection/train'
test_src = '../input/dog-breed-detection/test'

train_labels = pd.read_csv('../input/dog-breed-detection/labels.csv', index_col = 'id')
submission=pd.read_csv('../input/dog-breed-detection/sample_submission.csv')

In [6]:
train_size = len(os.listdir(train_src))
test_size = len(os.listdir(test_src))

print(train_size,test_size)
print(train_labels.shape)
print(submission.shape)

In [7]:
target, dog_breeds = pd.factorize(train_labels['breed'], sort = True)
train_labels['target'] = target

print(dog_breeds)

In [8]:
display(train_labels.head())
display(submission.head())

In [9]:
train_labels['breed'].value_counts()

In [10]:
plt.figure(figsize=(15, 10))
train_labels['breed'].value_counts().plot(kind='bar')
plt.show()

## Prepare the Train and Test Data Directories

In [11]:
data_dir = pathlib.Path('/kaggle/working/data')
data_dir.mkdir()

In [12]:
train_dir = data_dir / 'train'
test_dir = data_dir / 'test'

train_dir.mkdir()
test_dir.mkdir()

In [13]:
## Create a directory for each dog breed classes
for breed in dog_breeds:
    breed_dir = train_dir / breed
    breed_dir.mkdir()

In [14]:
for root, dirs, files in os.walk(train_src):
    for file in files:
        imgName = file.split('.')[0]
        src_file = os.path.join(train_src, file)
        destination = os.path.join(train_dir, train_labels.loc[imgName, 'breed'])
        shutil.copy2(src_file, destination)

In [15]:
shutil.copytree(test_src, test_dir / 'images')

## Global Variables

In [16]:
N_EPOCHS = 10
BATCH_SIZE = 32
IMG_HEIGHT = 299
IMG_WIDTH = 299

## Prepare the Dataset

In [17]:
train_ds = image_dataset_from_directory(
  directory = train_dir,
  labels = 'inferred',
  label_mode='int',
  batch_size=BATCH_SIZE,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  shuffle = True,
  seed=1234,
  validation_split=0.2,
  subset="training",
)

In [18]:
class_names = train_ds.class_names
print(len(class_names))
print(class_names)

In [19]:
val_ds = image_dataset_from_directory(
  directory = train_dir,
  labels = 'inferred',
  label_mode='int',
  batch_size=BATCH_SIZE,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  shuffle = True,
  seed=1234,
  validation_split=0.2,
  subset="validation",
)

In [20]:
test_ds = image_dataset_from_directory(
  directory = test_dir,
  label_mode= None,
  batch_size=BATCH_SIZE,
  image_size=(IMG_HEIGHT, IMG_WIDTH),
  shuffle = False,
  seed=1234
)

## Visualize the Data

In [21]:
plt.figure(figsize=(20, 20))

for images, labels in train_ds.take(1):
#   print(labels)
  for i in range(16):
    ax = plt.subplot(4, 4, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(dog_breeds[labels[i]])
    plt.axis("off")

In [22]:
plt.figure(figsize=(20, 20))
for images, labels in val_ds.take(1):
#   print(labels)
  for i in range(16):
    ax = plt.subplot(4, 4, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(dog_breeds[labels[i]])
    plt.axis("off")

In [23]:
plt.figure(figsize=(20, 20))
for images in test_ds.take(1):
#   print(labels)
  for i in range(16):
    ax = plt.subplot(4, 4, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
#     plt.title(dog_breeds[labels[i]])
    plt.axis("off")


## Configure the dataset for performance

Let's make sure to use buffered prefetching, so we can yield data from disk without having I/O become blocking. These are two important methods we should use when loading data.

.cache() keeps the images in memory after they're loaded off disk during the first epoch.

.prefetch() overlaps data preprocessing and model execution while training.


In [24]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
# test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

## Data Augmentation layer

In [25]:
data_augmentation = Sequential(
  [
    preprocessing.RandomFlip("horizontal"),
    preprocessing.RandomRotation(0.1),
    preprocessing.RandomZoom(0.1),
  ]
)



Let's visualize what the first image of the first batch looks like after various random transformations:


In [26]:
plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
    first_image = images[0]
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        augmented_image = data_augmentation(
            tf.expand_dims(first_image, 0), training=True
        )
        plt.imshow(augmented_image[0].numpy().astype("int32"))
        plt.title(dog_breeds[labels[0]])
        plt.axis("off")

## Build a Model

In [35]:
base_model = Xception(weights='imagenet', include_top=False, input_shape=(299,299,3)) 
# display(base_model.summary())

# train only the top layers (which were randomly initialized)
# i.e. freeze all convolutional Xception layers
base_model.trainable = False

inputs = Input(shape=(299, 299, 3))
x = data_augmentation(inputs)
x = preprocess_input(x) ##  tf.keras.applications.xception.preprocess_input

# The base model contains batchnorm layers. We want to keep them in inference mode
# when we unfreeze the base model for fine-tuning, so we make sure that the
# base_model is running in inference mode here by passing `training=False`.
x = base_model(x, training=False)

x = GlobalAveragePooling2D()(x)

#     adding extra dense layer
#     x = Dense(1024, activation='relu')(x)
#     x = Dropout(.7)(x)
#     x = Dense(512, activation='relu')(x)

x = Dropout(.5)(x)
outputs = Dense(120, activation='softmax')(x)
model = Model(inputs, outputs)

display(model.summary())

**Training The Model**

In [36]:
optimizer = Adam(learning_rate=0.001)
model.compile(loss="sparse_categorical_crossentropy", metrics=['accuracy'], optimizer=optimizer)

In [37]:
EarlyStop_callback = EarlyStopping(min_delta=0.001, patience=10, restore_best_weights=True)

# # DECREASE LEARNING RATE EACH EPOCH
# annealer = LearningRateScheduler(lambda epoch: 1e-5 * 0.95 ** epoch, verbose=1)

# cb=[PlotLossesKeras(), annealer]

In [38]:
history = model.fit(
    train_ds,
    epochs=N_EPOCHS,
    validation_data=val_ds,
    callbacks=[EarlyStop_callback]
)

In [39]:
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(10, 10))

plt.subplot(2, 1, 1)
plt.plot(accuracy, label='Training Accuracy')
plt.plot(val_accuracy, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.xticks(list(range(20)))
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xticks(list(range(20)))
plt.xlabel('Epoch')
plt.show()

**Fine Tuning**

In [40]:
# Unfreeze the base_model. Note that it keeps running in inference mode
# since we passed `training=False` when calling it. This means that
# the batchnorm layers will not update their batch statistics.
# This prevents the batchnorm layers from undoing all the training
# we've done so far.
# base_model.trainable = True
# model.summary()

In [41]:
# print("Number of layers in the base model: ", len(base_model.layers))

In [42]:
# fine_tune_from = 100

# # Freeze all the layers before the `fine_tune_at` layer
# for layer in base_model.layers[:fine_tune_from]:
#     layer.trainable =  False

In [44]:
# model.compile(
#     optimizer=keras.optimizers.Adam(1e-5),  # Low learning rate
#     loss=keras.losses.BinaryCrossentropy(from_logits=True),
#     metrics=[keras.metrics.BinaryAccuracy()],
# )

In [ ]:
# FINE_TUNE_EPOCHS = 10
# fine_tune_history = model.fit(train_ds,
#                          epochs = FINE_TUNE_EPOCHS + history.epoch[-1],
#                          initial_epoch = history.epoch[-1] + 1,
#                          validation_data = val_ds)

In [ ]:
# accuracy += history_fine.history['accuracy']
# val_accuracy += history_fine.history['val_accuracy']

# loss += history_fine.history['loss']
# val_loss += history_fine.history['val_loss']

In [ ]:
# plt.figure(figsize=(8, 8))
# plt.subplot(2, 1, 1)
# plt.plot(acc, label='Training Accuracy')
# plt.plot(val_acc, label='Validation Accuracy')
# plt.ylim([0.8, 1])
# plt.plot([EPOCHS-1,EPOCHS-1],
#           plt.ylim(), label='Start Fine Tuning')
# plt.legend(loc='lower right')
# plt.title('Training and Validation Accuracy')

# plt.subplot(2, 1, 2)
# plt.plot(loss, label='Training Loss')
# plt.plot(val_loss, label='Validation Loss')
# plt.ylim([0, 1.0])
# plt.plot([EPOCHS-1,EPOCHS-1],
#          plt.ylim(), label='Start Fine Tuning')
# plt.legend(loc='upper right')
# plt.title('Training and Validation Loss')
# plt.xlabel('epoch')
# plt.show()

## Predict on Test Data

In [46]:
del train_ds, val_ds, train_labels

In [47]:
file_paths = test_ds.file_paths
# print(file_paths)

In [48]:
predictions = model.predict(
    test_ds,
    batch_size = BATCH_SIZE,
    verbose=1         
)

In [49]:
print(predictions.shape)
print(predictions)

In [58]:
plt.figure(figsize=(20, 20))
for images in test_ds.take(1):
#   print(labels)
  for i in range(16):
    ax = plt.subplot(4, 4, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
#     plt.title(dog_breeds[labels[i]])
    plt.axis("off")

In [68]:
for i in range(16):
    a=np.argmax(predictions[i])
    print(i+1,class_names[a])